# プリミティブ検出


平面や球などの単純な図形を検出する処理を「プリミティブ検出」と呼びます．
机の上に並べた対象物を検出したい場合，事前に平面を検出しておき，その部分の点群を削除してしまえば，
個々の物体を簡単に単離（検出）することができます．

このように身の回りの構造物の多くは，単純な数式で記述できるプリミティブ形状であることが多いため，
プリミティブ検出は，シーン理解のためのかなり強力な前処理として利用することができます．
本節では，RANSACを使ったプリミティブ検出アルゴリズムについて紹介します．

## 平面の検出
Open3Dによる平面検出について紹介します．
対象となる点群データは```tabletop_scene1.ply```です．読み込んで表示してみましょう．
このデータは机の上に3Dプリンタで出力した配置した球やうさぎなどを計測した点群です．


In [ ]:
import open3d as o3d
import numpy as np
import copy
from common3Dfunc import *
o3d.__version__

In [ ]:
pcd = o3d.io.read_point_cloud("../data/tabletop_scene.ply")
o3d.visualization.draw_geometries([pcd])


この点群から，平面検出によって，机の面を検出してみましょう．
Open3Dでは，PointCloudのメンバとして関数```segment_plane()```が用意されています．
実行してみましょう．

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.005,
                                         ransac_n=3,
                                         num_iterations=500)

[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

```segment_plane()```では，RANSACによる平面検出を実行します．
引数は次の通りです．
- distance_threshold
    - RANSACの「評価」処理で利用されます．平面のインライアとして判定するための距離のしきい値です．0.005を設定すると，平面から距離5mm以内の点をインライアとしてみなします．
- ransac_n
    - RANSACの「サンプリング」処理で利用されます．この点数から平面のパラメータを計算します．
- num_iteration
    - RANSACの「サンプリング」と「評価」の繰り返し回数です．

出力は次の通りです．
- plane_model
    - 平面パラメータ
- inliers
    - 元の点群における，平面上の点のインデクスのリスト
    

```inliers```には，平面上の点のインデクスが保存されていますので，これを使って結果を確認しましょう．
以下のコードでは，インデクスのリストを使って，点群を平面上のもの（赤色）と，それ以外に分けています．

In [ ]:
plane_cloud = pcd.select_by_index(inliers)
plane_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)

In [ ]:
o3d.visualization.draw_geometries([plane_cloud,outlier_cloud])


机を計測した点群のみが赤色に変更されており，うまく平面検出に成功したことがわかります．

では，ここで平面検出に使われた関数```segment_plane()```の処理の内容を解説します．

先にも触れたとおり，```segment_plane()```ではRANSACによって平面を検出しています．
3次元点を$(x,y,z)$とすると，平面は$ax+by+cz+d=0$で表すことができます．
したがって，平面検出をおこなうRANSACのモデルパラメータは$a,b,c,d$となります．

### サンプリング
点群からランダムに3点$p_0, p_1, p_2$を選択し，この3点によって作られる平面のパラメータ$p_{plane}=(a,b,c,d)$を算出します．
ベクトル$e_0 = p_1-p_0$, $e_1 = p_2-p_0$とし，これらの外積を正規化したベクトル$n = e_1 \times e_2 / |e_1 \times e_2|$が平面の法線方向である$a,b,c$です．
また，$d = (-n,p_0)$です．


### 評価
得られたモデルパラメータ$p_{plane}$の良さを評価します．
点群を構成する各点$p$と平面間の距離を計算します．
一般的には点と直線の距離は$|ax+by+cz+d|/\sqrt{a^2+b^2+c^2}$で計算できますが，この計算は$p$の末尾に1を追加した4次元ベクトル$p'=(p,1)$と$p_{plane}$の内積で表すことができます．

この距離が```distance_threshold```で設定したしきい値以下である場合は，インライアとしてカウントします．
すべての点に対して計算が終了したとき，モデルの当てはめの良さ```fitness```を（インライア点数）/（総点数）とします．また，インライアの平均誤差も記録しておきます．


```num_iteration```の回数の試行を繰り返し，```fitness```が最も高い$p_{plane}$を解とします．同率の```fitness```を持つ$p_{plane}$がある場合は，インライアの平均誤差が小さい方を採用します．また，インライアに属する点のインデクスのリストを作成し，出力します．

ここまでの処理のサンプルコードの実行方法は次の通りです．
``` bash
python o3d_ransac_plane.py
```

# 球の検出
検出するモデルパラメータを方程式で表現することができれば，前述したRANSACのアルゴリズムを修正することによって，別のプリミティブを検出することも可能です．
open3Dには，球を検出する関数が実装されていませんので，本節で実装してみましょう．

#### 球のパラメータの算出
まずは，RANSACの「サンプリング」について考えます．ここでは，点群からサンプリングした数点から，球のパラメータを求める必要があります．
球のパラメータは中心$(a,b,c)$と半径$r$の合計4パラメータで表現できます．
ここでは，点群からランダムにサンプリングされた4点をもとにして，球のパラメータを算出する方法を考えます．
最もオーソドックスな解き方は，球の方程式に4つの3次元点を代入して，連立方程式を解く方法です．
3次元点を$p_n = (x_n,y_n,z_n)$とすると，球の方程式は$(x_n-a)^2+(y_n-b)^2+(z_n-c)^2=r^2$です．

各点を方程式に代入し，以下のようにn番目の式からm番目の式を引いて整理すると，$a,b,c$に関する3つの連立一次式方程式が残ります．
具体的には，$n=0,1,2$，$m=3$とすると，次の通りです．

\begin{equation}
\left\{
\begin{array}{l}
(x_0-x_3)a+(y_0-y_3)b+(z_0-z_3)c=(x_0^2-x_3^2+y_0^2-y_3^2+z_0^2-z_3^2)/2 \\
(x_1-x_3)a+(y_1-y_3)b+(z_1-z_3)c=(x_1^2-x_3^2+y_1^2-y_3^2+z_1^2-z_3^2)/2 \\
(x_2-x_3)a+(y_2-y_3)b+(z_2-z_3)c=(x_2^2-x_3^2+y_2^2-y_3^2+z_2^2-z_3^2)/2 
\end{array}
\right.
\end{equation}


Numpyを使えば，```numpy.linalg.solve(A,B)```によって連立一次式方程式を解くことができます．
この関数の引数```A```は連立方程式の左辺の$a,b,c$の係数を並べた正方行列，```B```は右辺を縦に並べた列ベクトルです．
この機能を使って，$a,b,c$を算出し，最後に$r$を計算することにします．
実装例は以下の通りです．
この関数では入力された4点の3次元点```p0, p1, p2, p3```を通る球の方程式のパラメータ$(a,b,c,r)$変数```coeff```として出力します．

In [ ]:
def ComputeSphereCoefficient( p0, p1, p2, p3 ):
    """ 与えられた4点を通る球の方程式のパラメータ(a,b,c,r)を出力する．
        解が求まらない場合は，
    Args:
      p0,p1,p2,p3(numpy.ndarray): 4 points (x,y,z)
    Return:
      Sphere coefficients.
    """

    A = np.array([p0-p3,p1-p3,p2-p3])
    
    p3_2 = np.dot(p3,p3)
    b = np.array([(np.dot(p0,p0)-p3_2)/2,
                  (np.dot(p1,p1)-p3_2)/2,
                  (np.dot(p2,p2)-p3_2)/2])
    coeff = np.zeros(3)
    try:
        ans = np.linalg.solve(A,b)
    except:
        print( "!!Error!! Matrix rank is", np.linalg.matrix_rank(A) )
        print( "  Return", coeff )
        pass
    else:
        tmp = p0-ans
        r = np.sqrt( np.dot(tmp,tmp) )
        coeff = np.append(ans,r)

    return coeff

### 評価
次にRANSACの「評価」のステップについて考えましょう．
すでに求めた球のパラメータを使って，（インライア点数）/（総点数）で計算されるモデルの当てはめの良さ```fitness```や，インライアの平均誤差を算出します．

点群中の1点を$p_n = (x_n, y_n, z_n)$，中心$q = (a, b, c)$とすると，その間の距離$d_n=||p_n-q||_2$と半径$r$が一致していれば，$p_n$は球面上に位置しているといえます．
そこで，しきい値```distance_th```を使って，$|d_n-r|$が```distance_th```未満の$p_n$をインライアに判定することにします．
この実装例は以下の通りです．

In [ ]:
def EvaluateSphereCoefficient( pcd, coeff, distance_th=0.01 ):
    """ 球の方程式の係数の当てはまりの良さを評価する．
    Args:
      pcd(numpy.ndarray): Nx3 points
      coeff(numpy.ndarray): shpere coefficient. (a,b,c,r)
      distance_th(float):
    Returns:
      fitness: score [0-1]. larger is better
      inlier_dist: smaller is better
      inliers: indices of inliers
    """
    fitness = 0 # インライア点数/全点数
    inlier_dist = 0 #インライアの平均距離
    inliers = None #インライア点の番号セット
    
    dist = np.abs( np.linalg.norm( pcd - coeff[:3], axis=1 ) - coeff[3] )
    n_inlier = np.sum(dist<distance_th)
    if n_inlier != 0:
        fitness = n_inlier / pcd.shape[0]
        inlier_dist = np.sum((dist<distance_th)*dist)/n_inlier
        inliers = np.where(dist<distance_th)[0]
    
    return fitness, inlier_dist, inliers

上記のコードで重要な部分は，以下の行です．
```python
dist = np.abs( np.linalg.norm( pcd - coeff[:3], axis=1 ) - coeff[3] )
```
ここではnumpyのブロードキャストの機能を使って，点群中のすべての点に対する球面との距離を一気に計算しています．変数```dist```には点数$n$の配列になっており，各要素が対応するインデクスの距離値です．
pythonではfor文等の繰り返し演算が遅いことが知られていますので，ブロードキャストを多用することがおすすめです．

### 繰り返し演算の実装
球検出のための「サンプリング」と「評価」の実装が終わりましたので，あとはこれら２つのステップを繰り返しながら良いパラメータを探す処理を実装するだけです．球検出への入力する点群は，平面検出でアウトライア（＝平面以外）と判定された点群にします．

In [ ]:
pcd = outlier_cloud
np_pcd = np.asarray(pcd.points)

In [ ]:
# Parameters
ransac_n = 4 # 点群から選択する点数．球の場合は4．
num_iterations = 1000 # RANSACの試行回数
distance_th = 0.005 # モデルと点群の距離のしきい値
max_radius = 0.05 # 検出する球の半径の最大値

# 解の初期化
best_fitness = 0 # モデルの当てはめの良さ．インライア点数/全点数
best_inlier_dist = 10000.0 #インライア点の平均距離
best_inliers = None # 元の点群におけるインライアのインデクス
best_coeff = np.zeros(4) # モデルパラメータ

for n in range(num_iterations):
    c_id = np.random.choice( np_pcd.shape[0], 4, replace=False )
    coeff = ComputeSphereCoefficient( np_pcd[c_id[0]], np_pcd[c_id[1]], np_pcd[c_id[2]], np_pcd[c_id[3]] )
    if max_radius < coeff[3]:
        continue
    fitness, inlier_dist, inliers = EvaluateSphereCoefficient( np_pcd, coeff, distance_th )
    if (best_fitness < fitness) or ((best_fitness == fitness) and (inlier_dist<best_inlier_dist)):
        best_fitness = fitness
        best_inlier_dist = inlier_dist
        best_inliers = inliers
        best_coeff = coeff
        print(f"Update: Fitness = {best_fitness:.4f}, Inlier_dist = {best_inlier_dist:.4f}")

if best_coeff.any() != False:
    print(f"Sphere equation: (x-{best_coeff[0]:.2f})^2 + (y-{best_coeff[1]:.2f})^2 + (z-{best_coeff[2]:.2f})^2 = {best_coeff[3]:.2f}^2")
else:
    print(f"No sphere detected.")

上記のコードでは，より高い```fitness```を持つパラメータが発見できるたびに解を更新しています．
```fitness```が同一の場合は，```inlier_dist```がより小さい方の解を採用しています．

では，インライア（=球と判定された点群）を青に着色して可視化してみましょう．

In [ ]:
sphere_cloud = pcd.select_by_index(best_inliers)
sphere_cloud.paint_uniform_color([0, 0, 1.0])
outlier_cloud = pcd.select_by_index(best_inliers, invert=True)
o3d.visualization.draw_geometries([sphere_cloud,outlier_cloud])


球を計測した点群の色が青になったことがわかります．
せっかくなので，算出したパラメータを元に，球のグラフィックを描画してみましょう．

In [ ]:
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=best_coeff[3])
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.3, 0.3, 0.7])
mesh_sphere.translate(best_coeff[:3])
o3d.visualization.draw_geometries([mesh_sphere]+[sphere_cloud+plane_cloud+outlier_cloud])



RANSACを使うことによって，平面や球などの基本的な図形を点群から検出することができました．
プリミティブ図形としては他にも円柱や円錐等があります．
実際にPCLやMATLABには円柱を検出するための関数が用意されています．